In [1]:
from Train import train
from DynamicsModel import model as DynamicsModel
from env1.RealisticPendulum import RealisticPendulum as env1
import jax 
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import orbax.checkpoint as ocp
import os 


ModuleNotFoundError: No module named 'env1.env1'

In [ ]:
# Load Data:
with jnp.load('Data/data.npz') as data:
    X_data1 = data['observation_t']
    X_data2 = data['action_t']
    X_data = jnp.concatenate((X_data1, X_data2), axis=1)
    y_data = data['observation_tp']
    print(X_data.shape, y_data.shape)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)
print(X_data[:10,:], y_data[:10,:])
rng = jax.random.PRNGKey(0)

(1782000, 5) (1782000, 4)
[[-4.402749   -3.3945      0.77425027 -3.7951906  -2.85725   ]
 [-4.3489575  -3.4808223   1.5320121  -1.6269966   2.1163223 ]
 [-4.2681317  -3.4908636   2.1815755   0.19445716  1.8793108 ]
 [-4.179244   -3.4706311   2.2353868   0.605529    0.08415771]
 [-4.108277   -3.4793782   1.6211463  -0.4862598  -1.9696319 ]
 [-4.024826   -3.4480648   2.2401025   1.1937723   1.8161194 ]
 [-3.933969   -3.3917592   2.2813146   1.4723125   0.07477856]
 [-3.8513706  -3.3486323   1.9925027   0.94445825 -0.9096937 ]
 [-3.7710836  -3.3058693   2.011596    1.1055242   0.0316143 ]
 [-3.7002308  -3.281488    1.6912901   0.44514883 -0.97768736]] [[-4.3489575  -3.4808223   1.5320121  -1.6269966 ]
 [-4.2681317  -3.4908636   2.1815755   0.19445716]
 [-4.179244   -3.4706311   2.2353868   0.605529  ]
 [-4.108277   -3.4793782   1.6211463  -0.4862598 ]
 [-4.024826   -3.4480648   2.2401025   1.1937723 ]
 [-3.933969   -3.3917592   2.2813146   1.4723125 ]
 [-3.8513706  -3.3486323   1.9925027 

In [ ]:
params = train(rng, X_train, y_train, X_test, y_test, observation_size = 4,action_size = 1, learning_rate=0.0001, num_epochs=100, batch_size=64)

Epoch 0, Train Loss: 0.4437, Test Loss: 0.4431
Epoch 10, Train Loss: 0.3617, Test Loss: 0.3610
Epoch 20, Train Loss: 0.3609, Test Loss: 0.3603
Epoch 30, Train Loss: 0.3611, Test Loss: 0.3606


In [ ]:
from env1.env1 import InvertedPendulum as env1

env1 = env1()
key = jax.random.key(0)

State = env1.reset(key)
State1 = env1.step(State, 1.0)
#print(State.pipeline_state)
#State.pipeline_state.q = jnp.array([0.0,0.0],dtype=jnp.float32)
qqd = jnp.concatenate((jnp.expand_dims(State.pipeline_state.q,axis=1),jnp.expand_dims(State.pipeline_state.qd, axis=1)), axis=0)[:,0]
qqd10 = jnp.concatenate((jnp.expand_dims(State1.pipeline_state.q,axis=1),jnp.expand_dims(State1.pipeline_state.qd, axis=1)), axis=0)

model = DynamicsModel(4,5)
input = jnp.concatenate((qqd, jnp.array([1.0])), axis=0)
qqd11 = model.apply({"params":params}, input )
print(qqd10, qqd11)



11
[[-4.364456 ]
 [-3.306514 ]
 [ 2.0459676]
 [-0.8970223]] [-4.3654413 -3.3330982  1.7228411 -1.5903403]


In [ ]:
ckpt_dir = os.path.abspath('./params')
checkpointer = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())
path = ocp.test_utils.erase_and_create_empty(ckpt_dir)
checkpointer.save(path / str(1), args=ocp.args.StandardSave(params))
